In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import os
import re
import seaborn as sns

In [3]:

run_progress = []
base_path = "D:\\rl_soccer\\2vs0\\junio_2022\\"

for directory in os.listdir(base_path):
        run_progress.append(pd.read_csv(f"{base_path}\\{directory}\\progress.txt", delimiter="\t"))

run_progress = pd.concat(run_progress)


In [7]:
successful_games = np.where(run_progress["Success rate"])[0]
successful_game_count = np.sum(run_progress["Success rate"])

In [9]:
successful_game_count/run_progress.shape[0]

0.715036091060522

In [195]:
game_end = [a for a in np.where(progress["done"])[0]]
game_succ = [a for a in np.where(progress["done"]*progress["goal_H_A"] == 1)[0]]
game_failed = [a for a in np.where(progress["done"]*progress["goal_H_A"] == -1)[0]]

In [196]:
successful_games


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [197]:
progress.index

RangeIndex(start=0, stop=36699, step=1)

In [198]:
pitch_size = [(round(float(pitch_s[0][1:]),1), round(float(pitch_s[1][:-1]), 1))for pitch_s in pitch_size]

In [199]:
while game_failed != []:
    print(len(game_failed))
    final_ind = game_end.index(game_failed[-1])
    final = game_failed[-1]
    start_ind = game_end[final_ind-1]
    index_list = progress.index[start_ind+1:final+1]
    progress.drop(index_list, 0, inplace=True)
    game_end = [a for a in np.where(progress["done"]==1)[0]]
    game_failed = [a for a in np.where(progress["done"]*progress["goal_H_A"] == -1)[0]]

In [200]:
progress.columns


Index(['hit', 'ball_position_x', 'ball_position_y', 'repossessed',
       '2_hit_dist', 'interception', 'last_hit', 'vel_0', 'vel_1',
       'received_pass_0', 'received_pass_1', 'player_0_pos_x',
       'player_1_pos_x', 'player_0_pos_y', 'player_1_pos_y', 'ball_out',
       'hit_H_A', 'goal_H_A', 'done'],
      dtype='object')

In [201]:
hit_d = [a for a in np.where(progress["hit_H_A"]==1)[0]]

In [202]:
progress["last_hit"]

0               -1
1               -1
2               -1
3               -1
4               -1
           ...    
36694    HOME_id=1
36695    HOME_id=1
36696    HOME_id=1
36697    HOME_id=1
36698    HOME_id=1
Name: last_hit, Length: 36699, dtype: object

In [203]:
hits = [a for a in np.where(progress["hit"]==1)[0]]

In [204]:
ball_out = [a for a in np.where(progress["goal_H_A"]*(1-progress["done"])==1)[0]]

In [205]:
score_x0 = []
score_y0 = []
score_x1 = []
score_y1 = []
x0_pos = []
y0_pos = []
x1_pos = []
y1_pos = []
xb_pos = []
yb_pos = []
pitch_x = 24
pitch_y = 18
prev_index = 0
for game_n, index in enumerate(game_end):
    px, py = pitch_size[game_n]

    if progress["last_hit"].iloc[index] == "HOME_id=0":
        score_x0 += [pitch_x*np.round(progress["player_0_pos_x"].iloc[index-1],1)/px]
        score_y0 += [pitch_y*np.round(progress["player_0_pos_y"].iloc[index-1],1)/py]
    elif progress["last_hit"].iloc[index] == "HOME_id=1":
        score_x1 += [pitch_x*np.round(progress["player_1_pos_x"].iloc[index],1)/px]
        score_y1 += [pitch_y*np.round(progress["player_1_pos_y"].iloc[index],1)/py]
    x0_pos += list(pitch_x*np.round(progress["player_0_pos_x"].iloc[index-1:index+1],1)/px)
    y0_pos +=  list(pitch_y*np.round(progress["player_0_pos_y"].iloc[index-1:index+1],1)/py)
    x1_pos += list(pitch_x*np.round(progress["player_1_pos_x"].iloc[index-1:index+1],1)/px)
    y1_pos +=  list(pitch_y*np.round(progress["player_1_pos_y"].iloc[index-1:index+1],1)/py)
    xb_pos += list(pitch_x*np.round(progress['ball_position_x'].iloc[index-1:index+1],1)/px)
    yb_pos +=  list(pitch_y*np.round(progress['ball_position_y'].iloc[index-1:index+1],1)/py)
    prev_index = index+1
x0_pos = np.array(x0_pos)[..., np.newaxis]
y0_pos = np.array(y0_pos)[..., np.newaxis]
x1_pos = np.array(x1_pos)[..., np.newaxis]
y1_pos = np.array(y1_pos)[..., np.newaxis]
xb_pos = np.array(xb_pos)[..., np.newaxis]
yb_pos = np.array(yb_pos)[..., np.newaxis]



IndexError: list index out of range

In [ ]:
progress.reset_index(drop=True, inplace=True)

In [ ]:
import seaborn as sns

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.image as mpimg

colors = [(0,0,0), (0, 1, 1), (0, 1, 0.75), (0, 1, 0), (0.75, 1, 0),
          (1, 1, 0), (1, 0.8, 0), (1, 0.7, 0), (1, 0, 0)]
img = mpimg.imread('futbol_field.png')

x_pos = [x0_pos, x1_pos, score_x0, score_x1, xb_pos]
y_pos = [y0_pos, y1_pos, score_y0, score_y1, yb_pos]
labels = ["p0_pos", "p1_pos", "p0_score_pos", "p1_score_pos", "ball_pos_at_score"]
position = zip(x_pos, y_pos)
#cm = LinearSegmentedColormap.from_list('sample', colors)
fig=plt.figure(figsize=(10,7))
columns = 3
rows = 2
for i, x in enumerate(position):
    fig.add_subplot(rows, columns, i+1)
    plt.imshow(img.transpose(1,0,2),  origin='lower', extent=[-18, 18, -24, 24])
    plt.scatter(x[1], x[0], color=colors[i])
    plt.title(labels[i])
plt.show()
'''
fig, ax = plt.subplots(3, 2,figsize=(10, 15))
ax[0, 0].imshow(img.transpose(1,0,2),  origin='lower', extent=[-18, 18, -24, 24])
ax[0, 1].imshow(img.transpose(1,0,2),  origin='lower', extent=[-18, 18, -24, 24])
ax[1, 1].imshow(img.transpose(1,0,2),  origin='lower', extent=[-18, 18, -24, 24])
ax[1, 0].imshow(img.transpose(1,0,2),  origin='lower', extent=[-18, 18, -24, 24])
ax[2, 0].imshow(img.transpose(1,0,2),  origin='lower', extent=[-18, 18, -24, 24])
ax[0,0].scatter(y0_pos, x0_pos, color='b', label="player 0")
ax[0, 1].scatter(y1_pos, x1_pos, color='m', label="player 1")
ax[2, 0].scatter(yb_pos, xb_pos, color='r', label="ball pos at kick")
ax[1, 1].scatter(score_y0, score_x0, color='k', label="score pos p0")
ax[1, 0].scatter(score_y1, score_x1, color="c", label= "score pos p1")

plt.legend()
#plt.colorbar()
plt.show()
'''



In [ ]:
fig, ax = plt.subplots()
receivers = ["player 0", "player1"]
received_pass = [np.sum(progress["received_pass_0"]), np.sum(progress["received_pass_1"])]
total_pass = sum(received_pass)
received_pass = [pass_pi/total_pass for pass_pi in received_pass]
plt.bar(receivers, received_pass)
plt.title("Amount of Received passes")

In [ ]:
fig, ax = plt.subplots()
receivers = ["player 0", "player1"]
scored_goals = [len(score_x0), len(score_x1)]
print(scored_goals)
total_goals = sum(scored_goals)
print(total_goals)
scored_goals = [goal_pi/total_goals for goal_pi in scored_goals]
plt.bar(receivers, scored_goals)
plt.title("Amount of Goals Scored")

In [ ]:
plt.hist(progress["2_hit_dist"].iloc[np.where(progress["2_hit_dist"]>0)[0]], density=True)
plt.title("2 hit distance")
plt.xlabel("hit distance")

In [ ]:
diff =np.hstack([x0_pos, y0_pos]) - np.hstack([x1_pos, y1_pos])
diff2 = np.square(diff)
mse = np.sqrt(np.sum(diff2, axis=1))
plt.hist(mse, density=True)
plt.title("player distance in scored matches")
plt.xlabel("player distance")
plt.ylabel("frequency")

In [ ]:
x = np.arange(-24, 25, 1)
y = np.arange(-18, 19, 1)
xx, yy = np.meshgrid(x, y)
print(xx.shape, yy.shape)

In [ ]:
#volver el cálculo de la parte inferior matricial
progress.index

In [ ]:
import statistics
vel1 = np.zeros((yy.shape[0], xx.shape[1]))
freq1 = np.zeros((yy.shape[0], xx.shape[1]))
vel0 = np.zeros((yy.shape[0], xx.shape[1]))
freq0 = np.zeros((yy.shape[0], xx.shape[1]))
closest = lambda M, x: np.argmin(np.abs(x-M))

for p_i in progress.index:

    x1, y1 = progress.iloc[p_i]["player_1_pos_x"], progress.iloc[p_i]["player_1_pos_y"]
    x0, y0 = progress.iloc[p_i]["player_0_pos_x"], progress.iloc[p_i]["player_0_pos_y"]
    ix1, iy1 = closest(xx[0,:], x1), closest(yy[:, 0], y1)
    vel1[iy1, ix1] = statistics.mean([vel1[iy1, ix1], progress.iloc[p_i]["vel_1"]])
    freq1[iy1, ix1] += 1
    ix0, iy0 = closest(xx[0,:], x0), closest(yy[:, 0], y0)
    vel0[iy0, ix0] = statistics.mean([vel0[iy0, ix0], progress.iloc[p_i]["vel_0"]])
    freq0[iy0, ix0] += 1

freq1 /= len(progress.index)
freq0 /= len(progress.index)



In [ ]:
print(np.max(vel1), np.min(vel1))
print(np.max(vel0), np.min(vel0))

In [ ]:
freq1 = freq1/(np.max(freq1) - np.min(freq1))
freq0 = freq0/(np.max(freq0) - np.min(freq0))
vel1 = 2*(vel1-np.min(vel1))/(np.max(vel1) - np.min(vel1))-1
print(np.max(vel1), np.min(vel1))

In [ ]:

vel0 = 2*(vel0-np.min(vel0))/(np.max(vel0) - np.min(vel0))-1 
print(np.max(vel0), np.min(vel0))

#FIXME: 
1. PASAR CANCHA A BLANCO (listo)
2. verificar proporciones cancha
3. Reescalar entre 0 y 1. (listo)
4. ver mejor combinación de alphas. (listo)
5. campo con flechas. (listo)
6. dejar de la misma forma color maps (listo)
7. como proseguir en entrenamiento
8. Ver posibilidad de correr en docker.
quiver

In [ ]:

def generate_field(ax, pitch_size=[24, 18]):

    rg = 7.32/90
    rgs = (11 + 7.32)/90
    ra = 40.32/90
    rav = 16.5/60
    rac = 5.5/60

   

    #Pitch Outline & Centre Line
    plt.plot([-pitch_size[0],-pitch_size[0]],[-pitch_size[1],pitch_size[1]], color="black")
    plt.plot([-pitch_size[0], pitch_size[0]],[pitch_size[1],pitch_size[1]], color="black")
    plt.plot([pitch_size[0], pitch_size[0]],[-pitch_size[1],pitch_size[1]], color="black")
    plt.plot([-pitch_size[0], pitch_size[0]],[-pitch_size[1], -pitch_size[1]], color="black")
    plt.plot([0,0],[-pitch_size[1],pitch_size[1]], color="black")

    # circles of the center:
    centreCircle = plt.Circle((0,0), 0.1*pitch_size[1] ,color="black",fill=False)
    ax.add_patch(centreCircle)


    #Area grande Izquierda
    plt.plot([-pitch_size[0]+pitch_size[0]*rav, -pitch_size[0]+pitch_size[0]*rav], [-pitch_size[1]*ra, pitch_size[1]*ra], 
                color="black")
    plt.plot([-pitch_size[0], -pitch_size[0] + pitch_size[0]*rav],[-pitch_size[1]*ra, -pitch_size[1]*ra],color="black")
    plt.plot([-pitch_size[0], -pitch_size[0] + pitch_size[0]*rav],[pitch_size[1]*ra, pitch_size[1]*ra],color="black")



    #Area grande derecha
    plt.plot([pitch_size[0]-pitch_size[0]*rav, pitch_size[0]-pitch_size[0]*rav], [-pitch_size[1]*ra, pitch_size[1]*ra], 
                color="black")
    plt.plot([pitch_size[0], pitch_size[0] - pitch_size[0]*rav],[-pitch_size[1]*ra, -pitch_size[1]*ra],color="black")
    plt.plot([pitch_size[0], pitch_size[0] - pitch_size[0]*rav],[pitch_size[1]*ra, pitch_size[1]*ra],color="black")


    #Area Chica izquierda
    plt.plot([-pitch_size[0], -pitch_size[0] + rac*pitch_size[0]],[-pitch_size[1]*rav, -pitch_size[1]*rav],color="black")
    plt.plot([-pitch_size[0], -pitch_size[0] + rac*pitch_size[0]],[pitch_size[1]*rav, pitch_size[1]*rav],color="black")
    plt.plot([-pitch_size[0]+rac*pitch_size[0]]*2, [-pitch_size[1]*rav, pitch_size[1]*rav], color="black")

    #Area Chica derecha
    plt.plot([pitch_size[0], pitch_size[0] - rac*pitch_size[0]],[-pitch_size[1]*rav, -pitch_size[1]*rav],color="black")
    plt.plot([pitch_size[0], pitch_size[0] - rac*pitch_size[0]],[pitch_size[1]*rav, pitch_size[1]*rav],color="black")
    plt.plot([pitch_size[0]-rac*pitch_size[0]]*2, [-pitch_size[1]*rav, pitch_size[1]*rav], color="black")


    #Tidy Axes
    #plt.axis('off')

    #sns.kdeplot(data["Xstart"],data["Ystart"], shade=True,n_levels=50)
    #plt.ylim(-pitch_size[1], pitch_size[1])
    #plt.xlim(-pitch_size[0], pitch_size[0])




In [ ]:
vel0  = vel0*freq0
vel0[vel0==0.000] = np.nan
freq0[freq0==0.000] = np.nan

In [ ]:
vel1  = vel1*freq1
vel1[vel1==0.000] = np.nan
freq1[freq1==0.000] = np.nan

In [ ]:

#Create figure
fig=plt.figure()
fig.set_size_inches(7, 5)
ax=fig.add_subplot(1,1,1)
generate_field(ax)
#sns.kdeplot(progress["player_0_pos_x"], progress["player_0_pos_y"], shade="True", alpha=.5, color="red")
p = ax.pcolor(xx,yy,freq0, cmap = plt.get_cmap("Reds"), alpha=0.6)
q = ax.pcolor(xx, yy, vel0, cmap = plt.get_cmap("BrBG"), alpha=0.47)


vel_color = plt.colorbar(q)
freq_color = plt.colorbar(p)
freq_color.ax.set_title("frequency")
vel_color.ax.set_title("velocity")
plt.title("heatmap of position and velocity to ball player 0", y=-0.2)

plt.show()

# color value with maximum


In [ ]:

#Create figure
fig=plt.figure()
fig.set_size_inches(7, 5)
ax=fig.add_subplot(1,1,1)
generate_field(ax)
sns.kdeplot(progress["player_1_pos_x"], progress["player_1_pos_y"], shade="True", alpha=.5, color="red")
#p = ax.pcolor(xx,yy,freq0, cmap = plt.get_cmap("Reds"), alpha=0.6)
#q = ax.pcolor(xx, yy, vel0, cmap = plt.get_cmap("BrBG"), alpha=0.47)


#vel_color = plt.colorbar(q)
#freq_color = plt.colorbar(p)
#freq_color.ax.set_title("frequency")
#vel_color.ax.set_title("velocity")
plt.title("heatmap of position through out the game of player 1", y=-0.2)

plt.show()

# color value with maximum


In [ ]:

#Create figure
fig=plt.figure()
fig.set_size_inches(7, 5)
ax=fig.add_subplot(1,1,1)
generate_field(ax)
sns.kdeplot(progress["player_0_pos_x"], progress["player_0_pos_y"], shade="True", alpha=.5, color="red")
#p = ax.pcolor(xx,yy,freq0, cmap = plt.get_cmap("Reds"), alpha=0.6)
#q = ax.pcolor(xx, yy, vel0, cmap = plt.get_cmap("BrBG"), alpha=0.47)


#vel_color = plt.colorbar(q)
#freq_color = plt.colorbar(p)
#freq_color.ax.set_title("frequency")
#vel_color.ax.set_title("velocity")
plt.title("heatmap of position through out the game of player 0", y=-0.2)

plt.show()

# color value with maximum


In [ ]:
vel1[vel1==0.000] = np.nan
fig=plt.figure()
fig.set_size_inches(7, 5)
ax=fig.add_subplot(1,1,1)
generate_field(ax)
sns.kdeplot(progress["player_1_pos_x"], progress["player_1_pos_y"], shade="True", alpha=.5, color="red")
#p = ax.pcolor(xx, yy, freq1, cmap = plt.get_cmap("Reds"), alpha=0.6)
#q = ax.pcolor(xx, yy,vel1*freq1, cmap = plt.get_cmap("BrBG"), alpha=0.47)


freq_color = plt.colorbar(p)
vel_color = plt.colorbar(q)
freq_color.ax.set_title("frequency")
#vel_color.ax.set_title("velocity")
plt.title("heatmap of position and velocity to ball player 1", y=-0.2)

plt.show()

In [ ]:
position0 = np.concatenate([x0_pos, y0_pos], -1)
position1 = np.concatenate([x1_pos, y1_pos], -1)

In [ ]:
vel02d = position0[1, :] - position0[0:-1, :]
vel12d = position1[1, :] - position1[1:-1, :]

In [ ]:
g_index = 1
g_start = game_end[g_index-1]+1 if g_index != 0 else game_end[0]
g_end = game_end[g_index]

In [ ]:
vel02d[g_start:g_end-1, 0]

In [ ]:
fig=plt.figure()
fig.set_size_inches(7, 5)
ax=fig.add_subplot(1,1,1)
generate_field(ax)
ax.quiver(position0[g_start+ 1:g_end, 0], position0[g_start+1:g_end, 1], vel02d[g_start:g_end-1, 0], vel02d[g_start:g_end-1, 1], color="r")
ax.set_title("velocity of player 0")

In [ ]:

fig=plt.figure()
fig.set_size_inches(7, 5)
ax=fig.add_subplot(1,1,1)
generate_field(ax)
ax.quiver(position1[g_start+ 1:g_end, 0], position1[g_start+1:g_end, 1], vel12d[g_start:g_end-1, 0], vel12d[g_start:g_end-1, 1], color="r", units=)
ax.set_title("velocity of player 1")